# Masakhane - Reverse Machine Translation for African Languages (Using JoeyNMT)

> ## NB
>### - The purpose of this Notebook is to build models that translate African languages(target language) *into* English(source language). This will allow us to in future be able to make translations from one African language to the other. If you'd like to translate *from* English, please use [this](https://github.com/masakhane-io/masakhane-mt/blob/master/starter_notebook.ipynb) starter notebook instead.

>### - We call this reverse training because normally we build models that make translations from the source language(English) to the target language. But in this case we are doing the reverse; building models that make translations from the target language to the source(English)

## Note before beginning:
### - The idea is that you should be able to make minimal changes to this in order to get SOME result for your own translation corpus. 

### - The tl;dr: Go to the **"TODO"** comments which will tell you what to update to get up and running

### - If you actually want to have a clue what you're doing, read the text and peek at the links

### - With 100 epochs, it should take around 7 hours to run in Google Colab

### - Once you've gotten a result for your language, please attach and email your notebook that generated it to masakhanetranslation@gmail.com

### - If you care enough and get a chance, doing a brief background on your language would be amazing. See examples in  [(Martinus, 2019)](https://arxiv.org/abs/1906.05685)

## Retrieve your data & make a parallel corpus

If you are wanting to use the JW300 data referenced on the Masakhane website or in our GitHub repo, you can use `opus-tools` to convert the data into a convenient format. `opus_read` from that package provides a convenient tool for reading the native aligned XML files and to convert them to TMX format. The tool can also be used to fetch relevant files from OPUS on the fly and to filter the data as necessary. [Read the documentation](https://pypi.org/project/opustools-pkg/) for more details.

Once you have your corpus files in TMX format (an xml structure which will include the sentences in your target language and your source language in a single file), we recommend reading them into a pandas dataframe. Thankfully, Jade wrote a silly `tmx2dataframe` package which converts your tmx file to a pandas dataframe. 

# Notes:

This is a notebook for training a nr - en translation model using the JW300 dataset.

We train on a relatively large configuration, which uses the suggestions and increases some of the parameters. The model was trained for 23 epochs.

The results were
```
2021-10-10 05:59:31,826 - INFO - joeynmt.prediction -  dev bleu[13a]:  51.47 [Beam search decoding with beam size = 5 and alpha = 1.0]
2021-10-10 06:01:47,043 - INFO - joeynmt.prediction - test bleu[13a]:  57.22 [Beam search decoding with beam size = 5 and alpha = 1.0]
```

## Nonstandard Changes

While most of this notebook is the same as the original template notebook, we made a few differences, mainly with regards to the joeynmt installation, as we has some problems with that.

Instead of `git clone ... && cd ... && pip install .`, we do the following:

```
! pip install joeynmt
! pip install torch==1.9.0
! pip install --upgrade sacrebleu==1.5.1


# make the directories since we didn't clone
!mkdir -p joeynmt/configs
!mkdir -p joeynmt/data
!mkdir -p joeynmt/scripts

# get the build_vocab.py script

!cd joeynmt/scripts && wget https://raw.githubusercontent.com/joeynmt/joeynmt/master/scripts/build_vocab.py
```

The git clone approach should also work however.


------


We also do not use google colab, and as such the google drive specific code (e.g. mounting the drive) is commented out. This can be uncommented as needed


A final note, we did not run the training commands in the notebook, but rather in a terminal, as that was more stable and disconnections were less brutal. The output will be shown here, and it should be completely reproducible by running the commands in the notebook instead. The only commands we ran like this were the training procedure and the testing. 

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# TODO: Set your source and target languages. Keep in mind, these traditionally use language codes as found here:
# These will also become the suffix's of all vocab and corpus files used throughout
import os
source_language = "en"
target_language = "af" 
lc = False  # If True, lowercase the data.
seed = 42  # Random seed for shuffling.
tag = "mcb_af_to_en_1007_v2_larger" # Give a unique name to your folder - this is to ensure you don't rewrite any models you've already submitted

os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language
os.environ["tag"] = tag

# This will save it to a folder in our gdrive instead!
!mkdir -p "nlp_things/masakhane/$tgt-$src-$tag"
os.environ["gdrive_path"] = "nlp_things/masakhane/%s-%s-%s" % (target_language, source_language, tag)

In [5]:
!echo $gdrive_path

nlp_things/masakhane/af-en-mcb_af_to_en_1007_v2_larger


In [6]:
# Install opus-tools
! pip install opustools-pkg

     |████████████████████████████████| 80 kB 9.2 MB/s  eta 0:00:01


In [7]:
# Downloading our corpus
! opus_read -d JW300 -s $src -t $tgt -wm moses -w jw300.$src jw300.$tgt -q

# extract the corpus file

# I had to use this way around one because the file was af-en.
! gunzip JW300_latest_xml_$tgt-$src.xml.gz


Alignment file /proj/nlpl/data/OPUS/JW300/latest/xml/af-en.xml.gz not found. The following files are available for downloading:

  11 MB https://object.pouta.csc.fi/OPUS-JW300/v1c/xml/af-en.xml.gz
 124 MB https://object.pouta.csc.fi/OPUS-JW300/v1c/xml/af.zip
 274 MB https://object.pouta.csc.fi/OPUS-JW300/v1c/xml/en.zip

 409 MB Total size
./JW300_latest_xml_af-en.xml.gz ... 100% of 11 MB
./JW300_latest_xml_af.zip ... 100% of 124 MB
./JW300_latest_xml_en.zip ... 100% of 274 MB


In [8]:
# Download the global test set.
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
  
# And the specific test set for this language pair.
os.environ["trg"] = target_language 
os.environ["src"] = source_language 

! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.en 
! mv test.en-$trg.en test.en
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.$trg 
! mv test.en-$trg.$trg test.$trg

--2021-10-07 13:53:38--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 277791 (271K) [text/plain]
Saving to: ‘test.en-any.en’

test.en-any.en      100%[===================>] 271.28K  --.-KB/s    in 0.006s  

2021-10-07 13:53:39 (44.2 MB/s) - ‘test.en-any.en’ saved [277791/277791]

--2021-10-07 13:53:39--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-af.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 2

In [10]:
# Read the test data to filter from train and dev splits.
# Store english portion in set for quick filtering checks.
en_test_sents = set()
filter_test_sents = "test.en-any.en"
j = 0
with open(filter_test_sents) as f:
    for line in f:
        en_test_sents.add(line.strip())
        j += 1
print('Loaded {} global test sentences to filter from the training/dev data.'.format(j))

Loaded 3571 global test sentences to filter from the training/dev data.


In [12]:
import pandas as pd

# TMX file to dataframe
source_file = 'jw300.' + source_language
target_file = 'jw300.' + target_language

source = []
target = []
skip_lines = []  # Collect the line numbers of the source portion to skip the same lines for the target portion.
with open(source_file) as f:
    for i, line in enumerate(f):
        # Skip sentences that are contained in the test set.
        if line.strip() not in en_test_sents:
            source.append(line.strip())
        else:
            skip_lines.append(i)             
with open(target_file) as f:
    for j, line in enumerate(f):
        # Only add to corpus if corresponding source was not skipped.
        if j not in skip_lines:
            target.append(line.strip())
    
print('Loaded data and skipped {}/{} lines since contained in test set.'.format(len(skip_lines), i))
    
df = pd.DataFrame(zip(source, target), columns=['source_sentence', 'target_sentence'])
# if you get TypeError: data argument can't be an iterator is because of your zip version run this below
#df = pd.DataFrame(list(zip(source, target)), columns=['source_sentence', 'target_sentence'])
df.head(3)

Loaded data and skipped 5470/1104023 lines since contained in test set.


,source_sentence,target_sentence
0,‘ They Lifted Me Out of Deep Depression ’,‘ Hulle het my uit diep depressie gehelp ’
1,A woman from England wrote :,’ n Vrou van Engeland het geskryf :
2,“ Dear Sir :,“ Geagte Meneer :


## Pre-processing and export

It is generally a good idea to remove duplicate translations and conflicting translations from the corpus. In practice, these public corpora include some number of these that need to be cleaned.

In addition we will split our data into dev/test/train and export to the filesystem.

In [13]:
# drop duplicate translations
df_pp = df.drop_duplicates()

# drop conflicting translations
# (this is optional and something that you might want to comment out 
# depending on the size of your corpus)
df_pp.drop_duplicates(subset='source_sentence', inplace=True)
df_pp.drop_duplicates(subset='target_sentence', inplace=True)

# Shuffle the data to remove bias in dev set selection.
df_pp = df_pp.sample(frac=1, random_state=seed).reset_index(drop=True)

/home-mscluster/mbeukman/anaconda3/envs/nlp_q2/lib/python3.7/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [14]:
# Install fuzzy wuzzy to remove "almost duplicate" sentences in the
# test and training sets.
! pip install fuzzywuzzy
! pip install python-Levenshtein
import time
from fuzzywuzzy import process
import numpy as np
from os import cpu_count
from functools import partial
from multiprocessing import Pool


# reset the index of the training set after previous filtering
df_pp.reset_index(drop=False, inplace=True)

# Remove samples from the training data set if they "almost overlap" with the
# samples in the test set.

# Filtering function. Adjust pad to narrow down the candidate matches to
# within a certain length of characters of the given sample.
def fuzzfilter(sample, candidates, pad):
    candidates = [x for x in candidates if len(x) <= len(sample)+pad and len(x) >= len(sample)-pad] 
      if len(candidates) > 0:
        return process.extractOne(sample, candidates)[1]
      else:
        return np.nan

  Using cached python-Levenshtein-0.12.2.tar.gz (50 kB)
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=171679 sha256=d35e14a3f1162e26f294203aaab2e486581dff940827f2c9a2258e1339642115
  Stored in directory: /home-mscluster/mbeukman/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built python-Levenshtein


In [15]:
# start_time = time.time()
# ### iterating over pandas dataframe rows is not recomended, let use multi processing to apply the function

# with Pool(cpu_count()-1) as pool:
#     scores = pool.map(partial(fuzzfilter, candidates=list(en_test_sents), pad=5), df_pp['source_sentence'])
# hours, rem = divmod(time.time() - start_time, 3600)
# minutes, seconds = divmod(rem, 60)
# print("done in {}h:{}min:{}seconds".format(hours, minutes, seconds))

# # Filter out "almost overlapping samples"
# df_pp = df_pp.assign(scores=scores)
# df_pp = df_pp[df_pp['scores'] < 95]

In [16]:
# This section does the split between train/dev for the parallel corpora then saves them as separate files
# We use 1000 dev test and the given test set.
import csv

# Do the split between dev/train and create parallel corpora
num_dev_patterns = 1000

# Optional: lower case the corpora - this will make it easier to generalize, but without proper casing.
if lc:  # Julia: making lowercasing optional
    df_pp["source_sentence"] = df_pp["source_sentence"].str.lower()
    df_pp["target_sentence"] = df_pp["target_sentence"].str.lower()

# Julia: test sets are already generated
dev = df_pp.tail(num_dev_patterns) # Herman: Error in original
stripped = df_pp.drop(df_pp.tail(num_dev_patterns).index)

with open("train."+source_language, "w") as src_file, open("train."+target_language, "w") as trg_file:
  for index, row in stripped.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")
    
with open("dev."+source_language, "w") as src_file, open("dev."+target_language, "w") as trg_file:
  for index, row in dev.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")

#stripped[["source_sentence"]].to_csv("train."+source_language, header=False, index=False)  # Herman: Added `header=False` everywhere
#stripped[["target_sentence"]].to_csv("train."+target_language, header=False, index=False)  # Julia: Problematic handling of quotation marks.

#dev[["source_sentence"]].to_csv("dev."+source_language, header=False, index=False)
#dev[["target_sentence"]].to_csv("dev."+target_language, header=False, index=False)

# Doublecheck the format below. There should be no extra quotation marks or weird characters.
! head train.*
! head dev.*

==> train.af <==
Moderne tegnologie stel leerders en studente in staat om makliker en op bedrewener wyse te kul .
Ons het eers gaan sit en die koste bereken .
□ Wat leer die vernietiging van Sodom en Gomorra ons omtrent Jehovah se geregtigheid ?
1948
Die taai kakkerlak
Jesus het groot mag in die hemel .
14 Jehovah het ’ n vasgestelde tyd bepaal vir sy enigverwekte Seun om as die Messias aarde toe te kom .
Ja , hulle predikingswerk is nie bloot ’ n roetine nie .
14 DIE BYBEL SE BESKOUINGStiptelikheid
As jy ’ n ongepaste emosionele band met iemand van die teenoorgestelde geslag smee , verswak dit jou band met jou huweliksmaat .

==> train.en <==
Modern technology enables students to cheat with new levels of ease and sophistication .
First , we sat down to count the cost .
□ What does the destruction of Sodom and Gomorrah teach us about Jehovah ’ s justice ?
1948
The Enduring Cockroach
Jesus has great power in heaven .
14 Jehovah had set a fixed time for his only-begotten Son to come to t



---


## Installation of JoeyNMT

JoeyNMT is a simple, minimalist NMT package which is useful for learning and teaching. Check out the documentation for JoeyNMT [here](https://joeynmt.readthedocs.io)  

In [ ]:
# I had trouble with joey git installation, so I just installed it through here, and made the requisite directories. But theoretically this should work with the github installation as in the original notebook.

# Install using pip
! pip install joeynmt
! pip install torch==1.9.0
! pip install --upgrade sacrebleu==1.5.1


!mkdir -p joeynmt/configs
!mkdir -p joeynmt/data
!mkdir -p joeynmt/scripts

!cd joeynmt/scripts && wget https://raw.githubusercontent.com/joeynmt/joeynmt/master/scripts/build_vocab.py

# Preprocessing the Data into Subword BPE Tokens

- One of the most powerful improvements for agglutinative languages (a feature of most Bantu languages) is using BPE tokenization [ (Sennrich, 2015) ](https://arxiv.org/abs/1508.07909).

- It was also shown that by optimizing the umber of BPE codes we significantly improve results for low-resourced languages [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021) [(Martinus, 2019)](https://arxiv.org/abs/1906.05685)

- Below we have the scripts for doing BPE tokenization of our data. We use 4000 tokens as recommended by [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021). You do not need to change anything. Simply running the below will be suitable. 

In [36]:
# One of the huge boosts in NMT performance was to use a different method of tokenizing. 
# Usually, NMT would tokenize by words. However, using a method called BPE gave amazing boosts to performance

# Do subword NMT
from os import path
os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language

# Learn BPEs on the training data.
os.environ["data_path"] = path.join("joeynmt", "data",target_language + source_language ) # Herman! 
! subword-nmt learn-joint-bpe-and-vocab --input train.$src train.$tgt -s 4000 -o bpe.codes.4000 --write-vocabulary vocab.$src vocab.$tgt

# Apply BPE splits to the development and test data.
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < train.$src > train.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < train.$tgt > train.bpe.$tgt

! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < dev.$src > dev.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < dev.$tgt > dev.bpe.$tgt
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < test.$src > test.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < test.$tgt > test.bpe.$tgt

# Create directory, move everyone we care about to the correct location
! mkdir -p $data_path
! cp train.* $data_path
! cp test.* $data_path
! cp dev.* $data_path
! cp bpe.codes.4000 $data_path
! ls $data_path

# Also move everything we care about to a mounted location in google drive (relevant if running in colab) at gdrive_path
! cp train.* "$gdrive_path"
! cp test.* "$gdrive_path"
! cp dev.* "$gdrive_path"
! cp bpe.codes.4000 "$gdrive_path"
! ls "$gdrive_path"

# Create that vocab using build_vocab
! chmod 777 joeynmt/scripts/build_vocab.py
! joeynmt/scripts/build_vocab.py joeynmt/data/$tgt$src/train.bpe.$src joeynmt/data/$tgt$src/train.bpe.$tgt --output_path joeynmt/data/$tgt$src/vocab.txt

# Some output
! echo "BPE Afrikaans Sentences"
! tail -n 5 test.bpe.$tgt
! echo "Combined BPE Vocab"
! tail -n 10 joeynmt/data/$tgt$src/vocab.txt  # Herman

bpe.codes.4000	dev.bpe.en  test.af	 test.en	 train.bpe.af
dev.af		dev.en	    test.bpe.af  test.en-any.en  train.bpe.en
dev.bpe.af	scripts     test.bpe.en  train.af	 train.en
bpe.codes.4000	dev.bpe.en  test.bpe.af  test.en-any.en  train.bpe.en
dev.af		dev.en	    test.bpe.en  train.af	 train.en
dev.bpe.af	test.af     test.en	 train.bpe.af
BPE Afrikaans Sentences
Ek moes hom om@@ k@@ oop sodat hy sy oë sou sluit vir die maa@@ t@@ skappy se on@@ eer@@ likheid .
Ge@@ volglik was ek bekend as ’ n on@@ eer@@ like man . Toe ek in die waarheid kom , het ek gewe@@ ier om daar@@ mee voor@@ t te gaan , al het ek ’ n goeie sal@@ ar@@ is ver@@ dien .
Ek stel ’ n goeie voorbeeld vir my twee seuns , en ek het voor@@ regte in die gemeente gekry .
Bel@@ a@@ st@@ ing@@ ou@@ di@@ te@@ ur@@ s en ander met wie ek be@@ sig@@ heid doen , weet nou dat ek ’ n eer@@ like man is . ”
R@@ ut het na Israel ge@@ trek , waar sy die ware God kon aan@@ bid .
Combined BPE Vocab
̣@@
Ì@@
ṇ@@
ז@@
À@@
Α
ḥ
Ο@@
˜
·@@


In [32]:
# Also move everything we care about to a mounted location in google drive (relevant if running in colab) at gdrive_path
! cp train.* "$gdrive_path"
! cp test.* "$gdrive_path"
! cp dev.* "$gdrive_path"
! cp bpe.codes.4000 "$gdrive_path"
! ls "$gdrive_path"

bpe.codes.4000	dev.bpe.en  test.bpe.af  test.en-any.en  train.bpe.en
dev.af		dev.en	    test.bpe.en  train.af	 train.en
dev.bpe.af	test.af     test.en	 train.bpe.af


# Creating the JoeyNMT Config

JoeyNMT requires a yaml config. We provide a template below. We've also set a number of defaults with it, that you may play with!

- We used Transformer architecture 
- We set our dropout to reasonably high: 0.3 (recommended in  [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021))

Things worth playing with:
- The batch size (also recommended to change for low-resourced languages)
- The number of epochs (we've set it at 30 just so it runs in about an hour, for testing purposes)
- The decoder options (beam_size, alpha)
- Evaluation metrics (BLEU versus Crhf4)

In [85]:
# This creates the config file for our JoeyNMT system. It might seem overwhelming so we've provided a couple of useful parameters you'll need to update
# (You can of course play with all the parameters if you'd like!)

name = '%s%s' % (target_language, source_language)
# gdrive_path = os.environ["gdrive_path"]

# Create the config
config = """
name: "{target_language}{source_language}_reverse_transformer"

data:
    src: "{target_language}"
    trg: "{source_language}"
    train: "data/{name}/train.bpe"
    dev:   "data/{name}/dev.bpe"
    test:  "data/{name}/test.bpe"
    level: "bpe"
    lowercase: False
    max_sent_length: 100
    src_vocab: "data/{name}/vocab.txt"
    trg_vocab: "data/{name}/vocab.txt"

testing:
    beam_size: 5
    alpha: 1.0

training:
    load_model: "../nlp_things/masakhane/af-en-mcb_af_to_en_1007_v2_larger/models/enaf_reverse_transformer-large-30-epochs/125000.ckpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "noam"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 1000      # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 8192
    batch_type: "token"
    eval_batch_size: 3600
    eval_batch_type: "token"
    batch_multiplier: 1
    early_stopping_metric: "ppl"
    epochs: 100                  # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 5000          # TODO: Set to at least once per epoch.
    logging_freq: 100
    eval_metric: "bleu"
    model_dir: "models/{name}_reverse_transformer_michael_larger_30epochs-more-1008"
    overwrite: True              # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    max_output_length: 100
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
""".format(name=name, gdrive_path=os.environ["gdrive_path"], source_language=source_language, target_language=target_language)
with open("joeynmt/configs/transformer_reverse_{name}.yaml".format(name=name),'w') as f:
    f.write(config)

In [59]:
!echo "$gdrive_path/models/${src}${tgt}_reverse_transformer-large/5000.ckpt"

nlp_things/masakhane/af-en-mcb_af_to_en_1007_v2_larger/models/enaf_reverse_transformer-large/5000.ckpt


# Train the Model

This single line of joeynmt runs the training using the config we made above

In [ ]:
# Train the model
# You can press Ctrl-C to stop. And then run the next cell to save your checkpoints! 
!cd joeynmt; python3 -m joeynmt train configs/transformer_reverse_$tgt$src.yaml

2021-10-08 05:32:16,822 - INFO - root - Hello! This is Joey-NMT (version 1.3).
2021-10-08 05:32:16,867 - INFO - joeynmt.data - Loading training data...
2021-10-08 05:32:36,109 - INFO - joeynmt.data - Building vocabulary...
2021-10-08 05:32:36,365 - INFO - joeynmt.data - Loading dev data...
2021-10-08 05:32:36,472 - INFO - joeynmt.data - Loading test data...
2021-10-08 05:32:36,497 - INFO - joeynmt.data - Data loaded.
2021-10-08 05:32:36,497 - INFO - joeynmt.model - Building an encoder-decoder model...
2021-10-08 05:32:36,778 - INFO - joeynmt.model - Enc-dec model built.
2021-10-08 05:32:36,792 - INFO - joeynmt.training - Total params: 12223488
2021-10-08 05:32:39,458 - INFO - joeynmt.training - Loading model from ../nlp_things/masakhane/af-en-mcb_af_to_en_1007_v2_larger/models/enaf_reverse_transformer-large-30-epochs/125000.ckpt
2021-10-08 05:32:39,791 - INFO - joeynmt.helpers - cfg.name                           : afen_reverse_transformer
2021-10-08 05:32:39,791 - INFO - joeynmt.helpe

In [92]:
os.environ['FILE'] = f"{os.environ['gdrive_path']}/models/{os.environ['src']}{os.environ['tgt']}_reverse_transformer-large-30-epochs-and-20-more/"

!echo $FILE
!ls $FILE

nlp_things/masakhane/af-en-mcb_af_to_en_1007_v2_larger/models/enaf_reverse_transformer-large-30-epochs-and-20-more/
130000.hyps  160000.hyps  190000.hyps  210000.ckpt    tensorboard
135000.hyps  165000.hyps  195000.hyps  210000.hyps    train.log
140000.hyps  170000.hyps  200000.ckpt  best.ckpt      trg_vocab.txt
145000.hyps  175000.hyps  200000.hyps  config.yaml    validations.txt
150000.hyps  180000.hyps  205000.ckpt  latest.ckpt
155000.hyps  185000.hyps  205000.hyps  src_vocab.txt


In [78]:
os.environ['gdrive_path']

'nlp_things/masakhane/af-en-mcb_af_to_en_1007_v2_larger'

In [93]:
# Copy the created models from the notebook storage to google drive for persistant storage
!mkdir -p "$FILE"

!cp -r joeynmt/models/${tgt}${src}_reverse_transformer_michael_larger_30epochs-more-1008/* "$FILE"



In [94]:
# Output our validation accuracy
! cat "$FILE/validations.txt"

Steps: 130000	Loss: 36860.40234	PPL: 3.38220	bleu: 44.68351	LR: 0.00008667	*
Steps: 135000	Loss: 36715.71094	PPL: 3.36606	bleu: 44.52790	LR: 0.00008505	*
Steps: 140000	Loss: 36628.31641	PPL: 3.35635	bleu: 44.40698	LR: 0.00008352	*
Steps: 145000	Loss: 36507.67188	PPL: 3.34299	bleu: 44.44326	LR: 0.00008207	*
Steps: 150000	Loss: 36433.37500	PPL: 3.33479	bleu: 44.53037	LR: 0.00008069	*
Steps: 155000	Loss: 36232.90234	PPL: 3.31276	bleu: 44.60506	LR: 0.00007938	*
Steps: 160000	Loss: 36215.22266	PPL: 3.31083	bleu: 44.46702	LR: 0.00007813	*
Steps: 165000	Loss: 36132.12891	PPL: 3.30174	bleu: 44.74444	LR: 0.00007693	*
Steps: 170000	Loss: 36024.71875	PPL: 3.29004	bleu: 44.90198	LR: 0.00007579	*
Steps: 175000	Loss: 35937.86328	PPL: 3.28061	bleu: 44.77740	LR: 0.00007470	*
Steps: 180000	Loss: 35954.15234	PPL: 3.28237	bleu: 44.80381	LR: 0.00007366	
Steps: 185000	Loss: 35815.46484	PPL: 3.26736	bleu: 44.75359	LR: 0.00007265	*
Steps: 190000	Loss: 35751.30469	PPL: 3.26044	bleu: 44.88682	LR: 0.00007169	*


In [95]:
# Test our model
! cd joeynmt; python3 -m joeynmt test ../"$FILE/config.yaml"

2021-10-08 14:52:12,417 - INFO - root - Hello! This is Joey-NMT (version 1.3).
2021-10-08 14:52:12,418 - INFO - joeynmt.data - Building vocabulary...
2021-10-08 14:52:12,699 - INFO - joeynmt.data - Loading dev data...
2021-10-08 14:52:12,713 - INFO - joeynmt.data - Loading test data...
2021-10-08 14:52:12,739 - INFO - joeynmt.data - Data loaded.
2021-10-08 14:52:12,787 - INFO - joeynmt.prediction - Process device: cuda, n_gpu: 1, batch_size per device: 18000 (with beam_size)
2021-10-08 14:52:15,375 - INFO - joeynmt.model - Building an encoder-decoder model...
2021-10-08 14:52:15,608 - INFO - joeynmt.model - Enc-dec model built.
2021-10-08 14:52:15,714 - INFO - joeynmt.prediction - Decoding on dev set (data/afen/dev.bpe.en)...
2021-10-08 14:52:59,280 - WARNING - sacrebleu - That's 100 lines that end in a tokenized period ('.')
2021-10-08 14:52:59,281 - WARNING - sacrebleu - It looks like you forgot to detokenize your test data, which may hurt your score.
2021-10-08 14:52:59,281 - WARNIN

In [99]:
! ls "$FILE"

130000.hyps  160000.hyps  190000.hyps  210000.ckpt    tensorboard
135000.hyps  165000.hyps  195000.hyps  210000.hyps    test.log
140000.hyps  170000.hyps  200000.ckpt  best.ckpt      train.log
145000.hyps  175000.hyps  200000.hyps  config.yaml    trg_vocab.txt
150000.hyps  180000.hyps  205000.ckpt  latest.ckpt    validations.txt
155000.hyps  185000.hyps  205000.hyps  src_vocab.txt


# Try and experiment with the parameters => Make the model bigger, and train for less epochs.

In [100]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'

In [105]:
# This creates the config file for our JoeyNMT system. It might seem overwhelming so we've provided a couple of useful parameters you'll need to update
# (You can of course play with all the parameters if you'd like!)

name = '%s%s' % (target_language, source_language)
# gdrive_path = os.environ["gdrive_path"]

# Create the config
config = """
name: "{target_language}{source_language}_reverse_transformer"

data:
    src: "{target_language}"
    trg: "{source_language}"
    train: "data/{name}/train.bpe"
    dev:   "data/{name}/dev.bpe"
    test:  "data/{name}/test.bpe"
    level: "bpe"
    lowercase: False
    max_sent_length: 100
    src_vocab: "data/{name}/vocab.txt"
    trg_vocab: "data/{name}/vocab.txt"

testing:
    beam_size: 5
    alpha: 1.0

training:
    load_model: "joeynmt/models/${tgt}${src}_reverse_transformer_big-model-end1008/"
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 1000      # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 4096
    batch_type: "token"
    eval_batch_size: 3600
    eval_batch_type: "token"
    batch_multiplier: 1
    early_stopping_metric: "ppl"
    epochs: 20                  # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 5000          # TODO: Set to at least once per epoch.
    logging_freq: 100
    eval_metric: "bleu"
    model_dir: "models/{name}_reverse_transformer_big-model-end1008"
    overwrite: True              # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    max_output_length: 100
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 8             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 512   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 512         # TODO: Increase to 512 for larger data.
        ff_size: 2048            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 8              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 512    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 512         # TODO: Increase to 512 for larger data.
        ff_size: 2048            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
""".format(name=name, gdrive_path=os.environ["gdrive_path"], source_language=source_language, target_language=target_language)
with open("joeynmt/configs/transformer_reverse_{name}.yaml".format(name=name),'w') as f:
    f.write(config)

In [106]:
!cd joeynmt; python3 -m joeynmt train configs/transformer_reverse_$tgt$src.yaml

2021-10-08 15:08:59,840 - INFO - root - Hello! This is Joey-NMT (version 1.3).
2021-10-08 15:08:59,892 - INFO - joeynmt.data - Loading training data...
2021-10-08 15:09:19,421 - INFO - joeynmt.data - Building vocabulary...
2021-10-08 15:09:19,676 - INFO - joeynmt.data - Loading dev data...
2021-10-08 15:09:19,781 - INFO - joeynmt.data - Loading test data...
2021-10-08 15:09:19,806 - INFO - joeynmt.data - Data loaded.
2021-10-08 15:09:19,806 - INFO - joeynmt.model - Building an encoder-decoder model...
2021-10-08 15:09:20,573 - INFO - joeynmt.model - Enc-dec model built.
2021-10-08 15:09:20,585 - INFO - joeynmt.training - Total params: 46467072
2021-10-08 15:09:23,190 - INFO - joeynmt.helpers - cfg.name                           : afen_reverse_transformer
2021-10-08 15:09:23,190 - INFO - joeynmt.helpers - cfg.data.src                       : af
2021-10-08 15:09:23,190 - INFO - joeynmt.helpers - cfg.data.trg                       : en
2021-10-08 15:09:23,190 - INFO - joeynmt.helpers - cf

In [107]:
os.environ['FILE'] = f"{os.environ['gdrive_path']}/models/{os.environ['src']}{os.environ['tgt']}_bigger-model/"

!echo $FILE
!ls $FILE

nlp_things/masakhane/af-en-mcb_af_to_en_1007_v2_larger/models/enaf_bigger-model/
ls: cannot access 'nlp_things/masakhane/af-en-mcb_af_to_en_1007_v2_larger/models/enaf_bigger-model/': No such file or directory


In [112]:
# Copy the created models from the notebook storage to google drive for persistant storage
!mkdir -p "$FILE"

# !cp -r joeynmt/models/${tgt}${src}_reverse_transformer_michael_larger_30epochs/* "$FILE"
!cp -r joeynmt/models/${tgt}${src}_reverse_transformer_big-model-end1008/* "$FILE"



In [110]:
# Output our validation accuracy
! cat "$FILE/validations.txt"

Steps: 5000	Loss: 68910.39844	PPL: 9.75744	bleu: 22.70799	LR: 0.00030000	*
Steps: 10000	Loss: 52362.63672	PPL: 5.64628	bleu: 33.08803	LR: 0.00030000	*
Steps: 15000	Loss: 47018.39844	PPL: 4.73190	bleu: 36.29706	LR: 0.00030000	*
Steps: 20000	Loss: 42779.03516	PPL: 4.11312	bleu: 39.27454	LR: 0.00030000	*
Steps: 25000	Loss: 40448.24609	PPL: 3.80811	bleu: 40.12918	LR: 0.00030000	*
Steps: 30000	Loss: 38351.96875	PPL: 3.55315	bleu: 41.78984	LR: 0.00030000	*


In [111]:
# Test our model
! cd joeynmt; python3 -m joeynmt test ../"$FILE/config.yaml"

2021-10-08 19:06:38,237 - INFO - root - Hello! This is Joey-NMT (version 1.3).
2021-10-08 19:06:38,238 - INFO - joeynmt.data - Building vocabulary...
2021-10-08 19:06:38,528 - INFO - joeynmt.data - Loading dev data...
2021-10-08 19:06:38,541 - INFO - joeynmt.data - Loading test data...
2021-10-08 19:06:38,568 - INFO - joeynmt.data - Data loaded.
2021-10-08 19:06:38,613 - INFO - joeynmt.prediction - Process device: cuda, n_gpu: 1, batch_size per device: 18000 (with beam_size)
2021-10-08 19:06:41,391 - INFO - joeynmt.model - Building an encoder-decoder model...
2021-10-08 19:06:42,250 - INFO - joeynmt.model - Enc-dec model built.
2021-10-08 19:06:42,449 - INFO - joeynmt.prediction - Decoding on dev set (data/afen/dev.bpe.en)...
2021-10-08 19:08:20,774 - WARNING - sacrebleu - That's 100 lines that end in a tokenized period ('.')
2021-10-08 19:08:20,774 - WARNING - sacrebleu - It looks like you forgot to detokenize your test data, which may hurt your score.
2021-10-08 19:08:20,774 - WARNIN

In [113]:
!ls joeynmt/models/${tgt}${src}_reverse_transformer_big-model-end1008/30000.ckpt

10000.hyps  20000.hyps	30000.ckpt  best.ckpt	 src_vocab.txt	train.log
15000.hyps  25000.ckpt	30000.hyps  config.yaml  tensorboard	trg_vocab.txt
20000.ckpt  25000.hyps	5000.hyps   latest.ckpt  test.log	validations.txt


# Now Continue Training

For the above, we stopped the training, but saved a checkpoint. Here we simply load the checkpoint again.

We performed the actual training in a normal terminal instead of in the notebook, but the output was saved, and that will be shown now.

The config shown below here is the same as the above one, but it contains the load_model option too.

In [ ]:
# This creates the config file for our JoeyNMT system. It might seem overwhelming so we've provided a couple of useful parameters you'll need to update
# (You can of course play with all the parameters if you'd like!)

name = '%s%s' % (target_language, source_language)
# gdrive_path = os.environ["gdrive_path"]

# Create the config
config = """
name: "{target_language}{source_language}_reverse_transformer"

data:
    src: "{target_language}"
    trg: "{source_language}"
    train: "data/{name}/train.bpe"
    dev:   "data/{name}/dev.bpe"
    test:  "data/{name}/test.bpe"
    level: "bpe"
    lowercase: False
    max_sent_length: 100
    src_vocab: "data/{name}/vocab.txt"
    trg_vocab: "data/{name}/vocab.txt"

testing:
    beam_size: 5
    alpha: 1.0

training:
    load_model: "../nlp_things/masakhane/af-en-mcb_af_to_en_1007_v2_larger/models/enaf_bigger-model/30000.ckpt"
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 1000      # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 4096
    batch_type: "token"
    eval_batch_size: 3600
    eval_batch_type: "token"
    batch_multiplier: 1
    early_stopping_metric: "ppl"
    epochs: 20                  # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 5000          # TODO: Set to at least once per epoch.
    logging_freq: 100
    eval_metric: "bleu"
    model_dir: "models/{name}_reverse_transformer_big-model-end1008"
    overwrite: True              # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    max_output_length: 100
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 8             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 512   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 512         # TODO: Increase to 512 for larger data.
        ff_size: 2048            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 8              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 512    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 512         # TODO: Increase to 512 for larger data.
        ff_size: 2048            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
""".format(name=name, gdrive_path=os.environ["gdrive_path"], source_language=source_language, target_language=target_language)
with open("joeynmt/configs/transformer_reverse_{name}.yaml".format(name=name),'w') as f:
    f.write(config)

In [ ]:
!cd joeynmt; python3 -m joeynmt train configs/transformer_reverse_$tgt$src.yaml

In [1]:
!cat logs_run_20_epochs

2021-10-08 19:23:32,135 - INFO - root - Hello! This is Joey-NMT (version 1.3).
2021-10-08 19:23:32,188 - INFO - joeynmt.data - Loading training data...
2021-10-08 19:23:51,543 - INFO - joeynmt.data - Building vocabulary...
2021-10-08 19:23:51,802 - INFO - joeynmt.data - Loading dev data...
2021-10-08 19:23:51,911 - INFO - joeynmt.data - Loading test data...
2021-10-08 19:23:51,939 - INFO - joeynmt.data - Data loaded.
2021-10-08 19:23:51,939 - INFO - joeynmt.model - Building an encoder-decoder model...
2021-10-08 19:23:52,736 - INFO - joeynmt.model - Enc-dec model built.
2021-10-08 19:23:52,749 - INFO - joeynmt.training - Total params: 46467072
2021-10-08 19:23:55,350 - INFO - joeynmt.training - Loading model from ../nlp_things/masakhane/af-en-mcb_af_to_en_1007_v2_larger/models/enaf_bigger-model/30000.ckpt
2021-10-08 19:23:55,866 - INFO - joeynmt.helpers - cfg.name                           : afen_reverse_transformer
2021-10-08 19:23:55,866 - INFO - joeynmt.helpers - cfg.data.src       

In [4]:
os.environ['FILE'] = 'nlp_things/masakhane/train_long_large/models/after_20_epochs/'
! mkdir -p $FILE
! cp -r joeynmt/models/afen_reverse_transformer_big-model-end1008/* "$FILE"

In [7]:
! cat $FILE/validations.txt

Steps: 35000	Loss: 37326.53516	PPL: 3.43472	bleu: 42.65708	LR: 0.00030000	*
Steps: 40000	Loss: 36330.92578	PPL: 3.32351	bleu: 43.04933	LR: 0.00030000	*
Steps: 45000	Loss: 35266.66797	PPL: 3.20862	bleu: 44.00645	LR: 0.00030000	*
Steps: 50000	Loss: 34401.17969	PPL: 3.11812	bleu: 44.15336	LR: 0.00030000	*
Steps: 55000	Loss: 33955.31641	PPL: 3.07249	bleu: 44.80005	LR: 0.00030000	*
Steps: 60000	Loss: 33105.29688	PPL: 2.98736	bleu: 45.69988	LR: 0.00030000	*
Steps: 65000	Loss: 32428.82031	PPL: 2.92130	bleu: 46.10860	LR: 0.00030000	*
Steps: 70000	Loss: 32225.17578	PPL: 2.90170	bleu: 46.56415	LR: 0.00030000	*
Steps: 75000	Loss: 31921.34961	PPL: 2.87270	bleu: 46.65734	LR: 0.00030000	*
Steps: 80000	Loss: 31358.10742	PPL: 2.81970	bleu: 47.06222	LR: 0.00030000	*
Steps: 85000	Loss: 31065.81250	PPL: 2.79259	bleu: 47.65710	LR: 0.00030000	*
Steps: 90000	Loss: 30789.02539	PPL: 2.76715	bleu: 47.42006	LR: 0.00030000	*
Steps: 95000	Loss: 30523.22852	PPL: 2.74295	bleu: 47.44714	LR: 0.00030000	*
Steps: 10000

In [ ]:
! cd joeynmt; python3 -m joeynmt test ../"$FILE/config.yaml"

In [2]:
!cat log_test

2021-10-10 05:57:47,548 - INFO - root - Hello! This is Joey-NMT (version 1.3).
2021-10-10 05:57:47,550 - INFO - joeynmt.data - Building vocabulary...
2021-10-10 05:57:47,830 - INFO - joeynmt.data - Loading dev data...
2021-10-10 05:57:47,847 - INFO - joeynmt.data - Loading test data...
2021-10-10 05:57:47,874 - INFO - joeynmt.data - Data loaded.
2021-10-10 05:57:47,922 - INFO - joeynmt.prediction - Process device: cuda, n_gpu: 1, batch_size per device: 18000 (with beam_size)
2021-10-10 05:57:50,673 - INFO - joeynmt.model - Building an encoder-decoder model...
2021-10-10 05:57:51,476 - INFO - joeynmt.model - Enc-dec model built.
2021-10-10 05:57:51,664 - INFO - joeynmt.prediction - Decoding on dev set (data/afen/dev.bpe.en)...
2021-10-10 05:59:31,494 - WARNING - sacrebleu - That's 100 lines that end in a tokenized period ('.')
2021-10-10 05:59:31,495 - WARNING - sacrebleu - It looks like you forgot to detokenize your test data, which may hurt your score.
2021-10-10 05:59:31,495 - WARNIN